In [52]:
import numpy as np
import pandas as pd

In [53]:
data_train = pd.read_csv('train.csv')
data = pd.read_csv('sample_submission.csv')
data_train

,user_id,order_completed_at,cart
0,2,2015-03-22 09:25:46,399
1,2,2015-03-22 09:25:46,14
2,2,2015-03-22 09:25:46,198
3,2,2015-03-22 09:25:46,88
4,2,2015-03-22 09:25:46,157
...,...,...,...
3123059,12702,2020-09-03 23:45:45,441
3123060,12702,2020-09-03 23:45:45,92
3123061,12702,2020-09-03 23:45:45,431
3123062,12702,2020-09-03 23:45:45,24


In [54]:
data

,id,target
0,0;133,0
1,0;5,1
2,0;10,0
3,0;396,1
4,0;14,0
...,...,...
790444,19998;26,0
790445,19998;31,0
790446,19998;29,1
790447,19998;798,1


In [55]:
data_train = data_train.drop(columns='order_completed_at')
data_train

,user_id,cart
0,2,399
1,2,14
2,2,198
3,2,88
4,2,157
...,...,...
3123059,12702,441
3123060,12702,92
3123061,12702,431
3123062,12702,24


In [56]:
len(set(data_train['user_id']))

20000

In [57]:
data_train['cart'].max()

880

In [58]:
data_train.loc[3]

user_id     2
cart       88
Name: 3, dtype: int64

In [59]:
data_train[data_train['user_id'] == 2].value_counts()

user_id  cart
2        23      11
         57      10
         61       8
         82       7
         403      7
                 ..
         168      1
         203      1
         219      1
         377      1
         820      1
Name: count, Length: 74, dtype: int64

In [60]:
len(set(data_train['cart'][data_train['user_id'] == 380].values))

250

In [61]:
x = 0
u_id = 0
for i in range(20000):
	y = len(set(data_train['cart'][data_train['user_id'] == i].values))
	if y > x:
		x = y
		u_id = i
print(x, u_id)

250 380


Методы коллаборативной фильтрации: Это подход, который основан на анализе поведения пользователей и предполагает, что пользователи, которые в прошлом выбирали товары из похожих категорий, вероятно, будут выбирать их и в будущем. Модели, основанные на коллаборативной фильтрации, могут использовать методы, такие как матричные разложения или алгоритмы кластеризации, чтобы предсказать следующий заказ пользователя.

Методы классификации или регрессии: Можно рассматривать задачу как задачу классификации или регрессии, где каждый товар или категория товаров представлен как признак, а целевой переменной будет следующий заказ пользователя. Модели, такие как случайный лес, градиентный бустинг или нейронные сети, могут быть применены для обучения на этих данных и предсказания следующего заказа.

In [62]:
data_train

,user_id,cart
0,2,399
1,2,14
2,2,198
3,2,88
4,2,157
...,...,...
3123059,12702,441
3123060,12702,92
3123061,12702,431
3123062,12702,24


In [109]:
import numpy as np

In [110]:
user_id = np.sort(pd.unique(data_train['user_id']))
user_id

array([    0,     1,     2, ..., 19997, 19998, 19999])

In [111]:
cart = np.sort(pd.unique(data_train['cart']))
cart = list(map(str, cart))
cart

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '130',
 '131',
 '132',
 '133',
 '134',
 '135',
 '136',
 '137',
 '138'

In [142]:
new_data = pd.DataFrame(index=range(0, 20000), columns=range(0, 881))
new_data

,0,1,2,3,4,5,6,7,8,9,...,872,873,874,875,876,877,878,879,880,user_id
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19995
19996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19996
19997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19997
19998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19998


In [150]:
for i in range(20000):
  d = dict(data_train[data_train['user_id'] == i].value_counts('cart'))
  list_keys = list(d.keys())
  list_values = list(d.values())
  new_data.loc[i][list_keys] = list_values

In [151]:
new_data = new_data.fillna(0)
new_data['user_id'] = range(0, 20000)
new_data

/var/folders/yj/nhkf3nv505d5kh0dsk42f3ym0000gn/T/ipykernel_90228/476989622.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_data['user_id'] = range(0, 20000)


,0,1,2,3,4,5,6,7,8,9,...,872,873,874,875,876,877,878,879,880,user_id
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1,0,0,0,0,2,0,0,0,1,...,0,0,0,0,0,0,0,0,0,2
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
4,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,19995
19996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,19996
19997,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,19997
19998,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,19998


In [144]:
user_iter = dict(data_train[data_train['user_id'] == 1].value_counts())
user_iter = {key[1]: value for key, value in user_iter.items()}
# user_id = 1
# my_df.loc[1] = user_iter

In [145]:
key_1 = []
value_1 = []
for key, value in user_iter.items():
    key_1.append(key)
    value_1.append(value)
print(key_1)
print(value_1)

[55, 14, 88, 798, 169, 812, 86, 82, 149, 54, 19, 171, 204, 231, 302, 307, 421, 23, 803, 170]
[9, 3, 3, 3, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [148]:
my_df.loc[0][0] = 0
my_df


/var/folders/yj/nhkf3nv505d5kh0dsk42f3ym0000gn/T/ipykernel_90228/1269474246.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_df.loc[0][0] = 0


,0,1,2,3,4,5,6,7,8,9,...,872,873,874,875,876,877,878,879,880,user_id
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19995
19996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19996
19997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19997
19998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19998


In [104]:
for i in range(0, 20000):
    user_iter = dict(data_train[data_train['user_id'] == i].value_counts())
    user_iter = {key[1]: value for key, value in user_iter.items()}
    user_id = i  
    my_df.loc[i] = user_iter

my_df

KeyboardInterrupt: 

In [102]:
user_2 = dict(data_train[data_train['user_id'] == i].value_counts())


In [101]:
user_2 = {key[1]: value for key, value in user_2.items()}
user_2

{23: 11,
 57: 10,
 61: 8,
 82: 7,
 403: 7,
 430: 5,
 409: 5,
 382: 5,
 398: 4,
 15: 4,
 383: 4,
 804: 4,
 22: 4,
 197: 3,
 396: 3,
 67: 3,
 380: 3,
 87: 3,
 54: 2,
 134: 2,
 393: 2,
 376: 2,
 16: 2,
 199: 2,
 198: 2,
 17: 2,
 19: 2,
 165: 2,
 5: 2,
 402: 2,
 24: 2,
 157: 2,
 420: 2,
 88: 2,
 432: 2,
 41: 2,
 84: 2,
 29: 2,
 160: 2,
 412: 2,
 100: 2,
 743: 1,
 408: 1,
 399: 1,
 395: 1,
 431: 1,
 425: 1,
 392: 1,
 808: 1,
 384: 1,
 406: 1,
 0: 1,
 381: 1,
 379: 1,
 9: 1,
 14: 1,
 25: 1,
 42: 1,
 43: 1,
 55: 1,
 64: 1,
 85: 1,
 89: 1,
 99: 1,
 129: 1,
 135: 1,
 139: 1,
 158: 1,
 159: 1,
 168: 1,
 203: 1,
 219: 1,
 377: 1,
 820: 1}